# Using FAC toolbox

- The `swarmx.io` module contains the `MagData` class which provides access to the necessary data via `viresclient`
- The `swarmx.toolboxes.fac` module contains the `fac_single_sat` function which accepts data stored within a `MagData` object

In [ ]:
import datetime as dt
import numpy as np
from swarmx.toolboxes.fac import FacInputs, fac_single_sat

First decide a time period to use. This can be provided as a `datetime` object or as an ISO-8601 string (e.g. `2022-01-01T00:00:00`).

In [ ]:
start_time = dt.datetime(2022, 1, 1)
end_time = dt.datetime(2022, 1, 2)

Now prepare the data to use for the analysis. The `collection` can be chosen from one of the compatible magnetic data collections, and the `model` is a VirES-compatible model string (see [viresclient](https://viresclient.readthedocs.io/en/latest/available_parameters.html) for more information). For example, one may choose `CHAOS-Core` (describing the time-dependent main field up to spherical harmonic degree 20) or `CHAOS` (the full model, describing the core, crust, and magnetosphere). The progress bars can be disabled with `show_progress=False`, and the request can be processed synchronously (suitable for short requests) with `asynchronous=False`.

In [ ]:
inputs = FacInputs(
    collection="SW_OPER_MAGA_LR_1B",
    model="IGRF",
    start_time=start_time,
    end_time=end_time,
    viresclient_kwargs=dict(asynchronous=True, show_progress=True),
)

The data now contains everything we need to apply the FAC algorithm. This data is contained in the `.xarray` attribute:

In [ ]:
inputs.xarray

Now let's apply the FAC algorithm:

In [ ]:
fac_results = fac_single_sat(inputs)
fac_results

Note that the results are given at the midpoints of the input time, so the `fac` array is one element shorter than the inputs. We can pad it with a `NaN` to easily compare it along the same time axis as the inputs.

Let's make a copy of the xarray object above and and merge the FAC estimates into it:

In [ ]:
ds_derived_fac = inputs.xarray.copy()
ds_derived_fac = ds_derived_fac.assign(
    {"FAC": (("Timestamp",), np.append(fac_results["fac"], np.nan))}
)
ds_derived_fac

Now let's compare what we have derived with what the official FAC product provides

We can use viresclient directly to fetch those data:

In [ ]:
from viresclient import SwarmRequest

request = SwarmRequest()

request.set_collection("SW_OPER_FACATMS_2F")
request.set_products(
    measurements=["FAC", "FAC_Error", "Flags", "Flags_F", "Flags_B", "Flags_q"],
)
data = request.get_between(start_time, end_time)
ds_fac_product = data.as_xarray()
ds_fac_product

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(2, 1, figsize=(15, 5))
axes[0].plot(
    ds_fac_product["Timestamp"],
    ds_fac_product["FAC"],
    alpha=0.5,
    color="blue",
    label="Data product",
)
axes[1].plot(
    ds_fac_product["Timestamp"],
    ds_derived_fac["FAC"],
    alpha=0.5,
    color="red",
    label="Derived from toolbox",
)
axes[1].set_xlabel("Time")
axes[1].set_ylabel("FAC")
fig.legend();